In [3]:
import pandas as pd

data = pd.read_csv("result-data/2022-9.csv")
# Replace nan values with 0
data = data.fillna(value=0)

In [9]:
def WattMinsToKWh(watt_seconds):
    return watt_seconds / 60 / 1000

def IsTimeOffPeak(hour_minute):
    t = hour_minute.split(":")
    minute = int(t[0]) * 60 + int(t[1])

    if (minute >= 0 and minute <= (3 * 60 + 30)) or (minute >= (23*60 + 30)):
        return True
    else:
        return False

def PandasProcessMonth(year, month, start_day, end_day):
    series_list = []

    for _day in range(start_day, end_day+1):
        charged_on_peak = []
        charged_off_peak = []
        zappi_imported_on_peak = []
        zappi_imported_off_peak = []

        day_string = str(year) + "-" + str(month) + "-" + str(_day)

        day_of_interest = data['Date'].str.fullmatch(day_string) 
        my_data = pd.DataFrame(data[day_of_interest])

        for i in range(my_data.first_valid_index(), my_data.first_valid_index() + len(my_data)):
            imported = my_data['Imported'][i]
            zappi_diverted = my_data['Zappi diverted'][i]
            zappi_imported = min(imported, zappi_diverted)

            offPeak = IsTimeOffPeak(my_data['Time'][i])

            if offPeak:
                zappi_imported_off_peak.append(zappi_imported)
                charged_off_peak.append(zappi_diverted)
            else:
                zappi_imported_on_peak.append(zappi_imported)
                charged_on_peak.append(zappi_diverted)

        if sum(zappi_imported_off_peak) > 0:
            rec = {}
            rec["Date"] = day_string
            rec["Import Amount"] = WattMinsToKWh(sum(zappi_imported_off_peak))
            rec["Unit Cost"] = 7.5
            rec["Cost"] = rec["Import Amount"] * rec["Unit Cost"] / 100
            rec["kWh Charged"] = WattMinsToKWh(sum(charged_off_peak))
            series_list.append(pd.Series(rec))
        
        if sum(zappi_imported_on_peak) > 0:
            rec = {}
            rec["Date"] = day_string
            rec["Import Amount"] = WattMinsToKWh(sum(zappi_imported_on_peak))
            rec["Unit Cost"] = 30.83
            rec["Cost"] = rec["Import Amount"] * rec["Unit Cost"] / 100
            rec["kWh Charged"] = WattMinsToKWh(sum(charged_on_peak))
            series_list.append(pd.Series(rec))

    output_df = pd.concat(series_list, axis=1).transpose()

    total = {}
    total["Date"] = None
    total["Import Amount"] = output_df["Import Amount"].sum()
    total["Unit Cost"] = None
    total["Cost"] = output_df["Cost"].sum()
    total["kWh Charged"] = output_df["kWh Charged"].sum()
    s = pd.Series(total)

    pd.concat([output_df, s.to_frame()], ignore_index=True)

    return output_df

result_df = PandasProcessMonth(2022, 9, 1, 30)

In [10]:
display(result_df)

,Date,Import Amount,Unit Cost,Cost,kWh Charged
0,2022-9-5,0.228433,30.83,0.070426,2.362917
1,2022-9-7,0.159617,30.83,0.04921,0.957767
2,2022-9-9,0.1029,30.83,0.031724,1.708383
3,2022-9-10,18.91805,7.5,1.418854,18.91805
4,2022-9-11,0.117767,30.83,0.036307,0.773533
5,2022-9-12,0.261633,30.83,0.080662,4.1741
6,2022-9-13,0.0092,30.83,0.002836,0.010333
7,2022-9-14,0.376917,30.83,0.116203,8.499267
8,2022-9-15,0.241,30.83,0.0743,2.636617
9,2022-9-16,18.116433,7.5,1.358732,18.116433
